<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/HTLim/NLP_docsummarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [196]:
# import packages
import numpy as np

In [197]:
class Extraction():
    def __init__(self):
        self.sentences = []
        self.sens_token = []
        self.connection = []
        self.weight_graph = []
        self.init_score = []
        self.score = []

    def add_sentence(self, sentence):
        self.sentences.append(sentence)
    
    # 각 문장 토크나이징
    def tokenizer(self):
        sens_token = []
        for sen in self.sentences:
            token = sen.split()
            sens_token.append(token)

        self.sens_token = sens_token
        return sens_token

    # 자카드 유사도 측정
    def zacard(self, token1, token2):
        return len(set(token1).intersection(set(token2)))/len(set(token1).union(set(token2)))
    
    # 문장 간 유사도 그래프 생성
    def connect(self):
        connection = []
        for i, seni in enumerate(self.sens_token):
            edges = []
            for j, senj in enumerate(self.sens_token):
                if i == j:
                    edges.append(0)
                else:
                    edges.append(self.zacard(seni, senj))
            connection.append(edges)
        self.connection = connection
        return connection

    # 엣지 가중치 그래프 생성
    def edge_weight(self):
        sim_graph = np.array(self.connection)
        edge_graph = []
        init_score = []
        for one in sim_graph:
            add_all = one.sum()
            edge = one/add_all

            init_score.append(add_all)
            edge_graph.append(edge)

        self.weight_graph = edge_graph
        self.init_score = init_score
        return edge_graph, init_score

    # 중요도 계산
    def cal_weight(self, iter, stop):
        score = np.array(self.init_score)
        graph = np.array(self.weight_graph)

        pre_score = score
        
        for i in range(iter):

            score_product = np.dot(score, graph)
            front = 0.15
            back = score_product * 0.85
            score = front + back

            a = max(np.abs(pre_score - score))
            
            self.score = score
            
        return score

In [198]:
# sentence data
sen1 = '딸기 바나나 사과 파인애플 수박'
sen2 = '바나나 사과 딸기 포도'
sen3 = '복숭아 수박'
sen4 = '파인애플 사과 딸기 바나나'
sens = [sen1, sen2, sen3, sen4]

In [199]:
ex = Extraction()

In [200]:
ex.add_sentence(sen1)
ex.add_sentence(sen2)
ex.add_sentence(sen3)
ex.add_sentence(sen4)

In [201]:
ex.tokenizer()

[['딸기', '바나나', '사과', '파인애플', '수박'],
 ['바나나', '사과', '딸기', '포도'],
 ['복숭아', '수박'],
 ['파인애플', '사과', '딸기', '바나나']]

In [202]:
ex.connect()

[[0, 0.5, 0.16666666666666666, 0.8],
 [0.5, 0, 0.0, 0.6],
 [0.16666666666666666, 0.0, 0, 0.0],
 [0.8, 0.6, 0.0, 0]]

In [203]:
ex.edge_weight()

([array([0.        , 0.34090909, 0.11363636, 0.54545455]),
  array([0.45454545, 0.        , 0.        , 0.54545455]),
  array([1., 0., 0., 0.]),
  array([0.57142857, 0.42857143, 0.        , 0.        ])],
 [1.4666666666666668, 1.1, 0.16666666666666666, 1.4])

In [204]:
ex.cal_weight(20, 100)

array([1.41215533, 1.02593219, 0.28643249, 1.28064793])

In [205]:
ex.score

array([1.41215533, 1.02593219, 0.28643249, 1.28064793])